In [2]:
import numpy as np

In [3]:
import pandas as pd

df= pd.read_csv("Main_Craigslist.csv")



In [4]:
df.columns

Index(['posting_date', 'Unnamed: 0', 'id', 'url', 'region', 'region_url',
       'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders',
       'fuel', 'odometer', 'title_status', 'transmission', 'VIN', 'drive',
       'size', 'type', 'paint_color', 'image_url', 'description', 'state',
       'lat', 'long', 'posting_date.1', 'removal_date'],
      dtype='object')

In [5]:
df["posting_date"]= pd.to_datetime(df["posting_date"])

df.drop("description",inplace=True,axis=1)




In [6]:
# df.dtypes

df["removal_date"] = pd.to_datetime(df["removal_date"])
df["posting_date"]= pd.to_datetime(df["posting_date"])
df= df.astype({       'price':"float64", 'year':"int64","year":"int64"})

df.dtypes

posting_date      datetime64[ns, UTC]
Unnamed: 0                      int64
id                              int64
url                            object
region                         object
region_url                     object
price                         float64
year                            int64
manufacturer                   object
model                          object
condition                      object
cylinders                      object
fuel                           object
odometer                      float64
title_status                   object
transmission                   object
VIN                            object
drive                          object
size                           object
type                           object
paint_color                    object
image_url                      object
state                          object
lat                           float64
long                          float64
posting_date.1                 object
removal_date

In [7]:
df['transmission'].unique()

array(['automatic', 'other', 'manual'], dtype=object)

In [8]:
from random import choice
fuel = df["fuel"].unique()
fuels= np.delete(fuel,2)
df["fuel"].replace("other",choice(fuel),inplace=True)

print(df["fuel"].unique())

['gas' 'diesel' 'hybrid' 'electric']


In [13]:
# df.rename({'Unnamed: 0':"posting_id"},axis=1,inplace=True)

# df.rename({"id":"veh_id","year":"manufac_year","condition":"veh_condition","size":"veh_size","type":"veh_type","state":"veh_state","long":"longitude"},inplace=True,axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 27 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   posting_date    426880 non-null  datetime64[ns, UTC]
 1   Unnamed: 0      426880 non-null  int64              
 2   id              426880 non-null  int64              
 3   url             426880 non-null  object             
 4   region          426880 non-null  object             
 5   region_url      426880 non-null  object             
 6   price           426880 non-null  float64            
 7   year            426880 non-null  int64              
 8   manufacturer    426880 non-null  object             
 9   model           426880 non-null  object             
 10  condition       426880 non-null  object             
 11  cylinders       426880 non-null  object             
 12  fuel            426880 non-null  object             
 13  odometer      

In [9]:
df.drop("posting_date.1",axis=1,inplace=True)
df.columns

Index(['posting_date', 'Unnamed: 0', 'id', 'url', 'region', 'region_url',
       'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders',
       'fuel', 'odometer', 'title_status', 'transmission', 'VIN', 'drive',
       'size', 'type', 'paint_color', 'image_url', 'state', 'lat', 'long',
       'removal_date'],
      dtype='object')

In [70]:
labels = ["short","medium","long"]
bins=[df["stay_time"].min(),df["stay_time"].mean(),df["stay_time"].max() ]


df["stay_time"] = df["removal_date"] - df["posting_date"]
df.head().loc[:,["posting_date","removal_date","stay_time"]]

df["stay_length"] = pd.cut(df["stay_time"],bins=3,include_lowest=True,labels=labels)
df.head(1000).loc[:,["posting_date","removal_date","stay_time","stay_length"]]

df["stay_length"].value_counts()

stay_length
short     316773
medium    108862
long        1245
Name: count, dtype: int64

In [75]:
df.drop("stay_time",axis=1, inplace=True)
df.columns

Index(['posting_date', 'posting_id', 'veh_id', 'url', 'region', 'region_url',
       'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders',
       'fuel', 'odometer', 'title_status', 'transmission', 'VIN', 'drive',
       'size', 'type', 'paint_color', 'image_url', 'state', 'lat', 'long',
       'removal_date', 'stay_length'],
      dtype='object')

In [76]:
import psycopg2

conn = psycopg2.connect(
    dbname ="main_craigslist",
    password="@Liseveve97",
    host="localhost",
    user="postgres"
)

curr = conn.cursor()

# query2 = """
# ALTER TABLE LISTED_VEHICLES ADD stay_length VARCHAR(10)
# """

In [68]:
# curr.execute(query2

query3 = """
INSERT INTO LISTED_VEHICLES (stay_length) VALUES(%s)

"""
for stay_length in df["stay_length"]:
    curr.execute(query3, (stay_length,))
    
    
conn.commit()
curr.close()
conn.close()

KeyboardInterrupt: 

In [77]:
query = """
INSERT INTO LISTED_VEHICLES (
    posting_date, posting_id, veh_id, url, region, region_url, price, manufac_year, manufacturer,
    model, veh_condition, cylinders, fuel, odometer, title_status, transmission, VIN, drive,
    veh_size, veh_type, paint_color, image_url, veh_state, lat, longitude, removal_date,stay_length
)  VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

for i in df.itertuples(index=False):
    curr.execute(query,i)
    
conn.commit()
curr.close()
conn.close()

In [45]:
curr.close()
conn.close()


In [43]:
new_df = pd.read_clipboard()
new_df.head()

,model,manufacturer,manufac_year
0,f-150,chevrolet,1978
1,c10,chevrolet,1959
2,mustang,ford,1965
3,beetle,volkswagen,1970
4,ranger,ford,1992


In [36]:
import pyodbc

pyodbc.drivers()

import pyodbc

conn = pyodbc.connect(
    Driver="ODBC Driver 17 for SQL Server",
    Server='JAQENIZO\SQLEXPRESS',
    Trusted_Connection="yes",  # or "no" if not using Windows Authentication
    Database="CRAIGSLIST"
)


cursor = conn.cursor()







In [17]:
query4 = """
ALTER TABLE VEHICLES_DETAILS ADD stay_length VARCHAR(10)
"""

cursor.execute(query4)

In [19]:
query5 = '''
INSERT INTO VEHICLES_DETAILS (stay_length) VALUES (?)
'''

for stay_length in df["stay_length"]:
    cursor.execute(query5, (stay_length,))

conn.commit()

In [20]:
# conn.commit()
cursor.close()
conn.close

<function Connection.close>

In [35]:
# df.drop("stay_time",axis=1,inplace=True)
df.rename({'id':'veh_id' },axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   posting_date  426880 non-null  datetime64[ns, UTC]
 1   posting_id    426880 non-null  int64              
 2   veh_id        426880 non-null  int64              
 3   url           426880 non-null  object             
 4   region        426880 non-null  object             
 5   region_url    426880 non-null  object             
 6   price         426880 non-null  float64            
 7   year          426880 non-null  int64              
 8   manufacturer  426880 non-null  object             
 9   model         426880 non-null  object             
 10  condition     426880 non-null  object             
 11  cylinders     426880 non-null  object             
 12  fuel          426880 non-null  object             
 13  odometer      426880 non-null  float64      

In [37]:
insert_query = """
INSERT INTO VEHICLES_DETAILS(
    posting_date, posting_id, id, link_url, region, region_url, price, manufac_year, manufacturer,
    model, car_condition, cylinders, fuel, odometer, title_status, transmission, VIN, drive,
    size, car_type, paint_color, image_url, veh_state, lat, longitude, removal_date,stay_length
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?)
"""

for i in df.itertuples(index=False):
    conn.execute(insert_query,i)
    
conn.commit()
cursor.close()
conn.close()

In [43]:
df["transmission"].unique()

array(['automatic', 'other', 'manual'], dtype=object)

In [46]:
# pyodbc.drivers()

# conn2 = pyodbc.connect(
#     DRIVER={SQL Server},
#     Trusted_connection= 'Yes',
#     Database =""
    
# )

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'ODBC Driver 17 for SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'PostgreSQL ANSI',
 'PostgreSQL Unicode']